# Virtual Linking Bids for Market-Clearing with Non-Merchant Storage

This is the code for the paper "Virtual Linking Bids for Market-Clearing with Non-Merchant Storage". Different types of market clearing setups can be run and compared, including the examples presented in the paper. The code was written for Julia v1.11.

In [1]:
# Include the relevant packages
using CSV, DataFrames
using JuMP, HiGHS

The different parameters can be updated in the csv files "data_gen.csv" and "data_load.csv" and below:

In [2]:
## MARKET DATA
# Duration of time periods (in hours):
∆t = 1

## STORAGE DATA
# Storage capacity (MWh)
E = 2.5 
# Initial state of energy (MWh)
E_0 = 0.0 
# Efficiencies
η_C = 1.0
η_D = 1.0
# Max charge and discharge (MW)
P_C = 3.5
P_D = 3.5
;

In [3]:
# Dataframes are created from the csv files for generators:
G_df = CSV.read("data_gen.csv", DataFrame)

Row,MC,t,ID,max,cost
,Int64,Int64,Int64,Float64,Int64
1,1,1,1,1.0,0
2,1,2,1,2.5,2
3,1,3,1,0.0,0
4,2,1,1,2.0,3
5,2,2,1,1.0,2
6,2,3,1,5.5,3


In [4]:
# And loads:
L_df = CSV.read("data_load.csv", DataFrame)

Row,MC,t,ID,max,utility
,Int64,Int64,Int64,Float64,Int64
1,1,1,1,0.0,0
2,1,2,1,1.0,5
3,1,3,1,2.0,6
4,2,1,1,2.5,4
5,2,2,1,0.0,0
6,2,3,1,4.0,7


## Ideal Market Clearing

In [5]:
# Set the level for the storage system at the end of the test period:
E_final = 2.5
;

In [6]:
# Run the market clearing
include.(("fct_perfect.jl","fct_mc_all.jl","fct_data_all.jl"))
df_MCP = perfect(∆t, E, E_0, η_C, η_D, P_C, P_D, E_final, G_df, L_df)

SW_tot: 25.5
surplus_gen_tot: [3.0]
surplus_gen_sum_tot: 3.0
surplus_load_tot: [27.5]
surplus_stg_tot: -5.0


Row,Time,Market_Price,C1,p1,P1,U1,d1,D1,p_C,p_D,e
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,2.0,2.0,2.0,2.5,2.5,5.0,1.0,1.0,1.5,0.0,2.5
3,3.0,3.0,0.0,-0.0,0.0,6.0,2.0,2.0,0.0,2.0,0.5
4,4.0,3.0,3.0,2.0,2.0,4.0,2.5,2.5,0.0,0.5,-0.0
5,5.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
6,6.0,3.0,3.0,5.5,5.5,7.0,4.0,4.0,1.5,0.0,2.5


## Split Market Clearing with Level

In [7]:
# Set the level for the storage system at the end of each market interval:
E_end = [0.5, 2.5]
;

In [8]:
# Run the market clearing
include.(("fct_split.jl","fct_mc_all.jl","fct_data_all.jl"))
S_end = zeros(maximum(G_df.MC)) # Set the penalty to zero (no penalty)
df_MCS = split(∆t, E, E_0, η_C, η_D, P_C, P_D, G_df, L_df, E_end, S_end)

SW_tot: 25.5
surplus_gen_tot: [5.0]
surplus_gen_sum_tot: 5.0
surplus_load_tot: [19.0]
surplus_stg_tot: 1.5


Row,Time,Market_Price,C1,p1,P1,U1,d1,D1,p_C,p_D,e
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,2.0,2.0,2.0,2.5,2.5,5.0,1.0,1.0,1.5,0.0,2.5
3,3.0,6.0,0.0,-0.0,0.0,6.0,2.0,2.0,0.0,2.0,0.5
4,4.0,4.0,3.0,2.0,2.0,4.0,2.5,2.5,0.0,0.5,0.0
5,5.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
6,6.0,3.0,3.0,5.5,5.5,7.0,4.0,4.0,1.5,0.0,2.5


## Split Market Clearing with Penalty

In [9]:
# Set the penalty for the storage system at the end of each market interval:
S_end = [1.28, 0]
;

In [10]:
# Run the market clearing
include.(("fct_split.jl","fct_mc_all.jl","fct_data_all.jl"))
E_end = [false, false] # No final level
df_MCS = split(∆t, E, E_0, η_C, η_D, P_C, P_D, G_df, L_df, E_end, S_end)

SW_tot: 32.0
surplus_gen_tot: [5.0]
surplus_gen_sum_tot: 5.0
surplus_load_tot: [27.0]
surplus_stg_tot: 0.0


Row,Time,Market_Price,C1,p1,P1,U1,d1,D1,p_C,p_D,e
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,2.0,2.0,2.0,2.0,2.5,5.0,1.0,1.0,1.0,0.0,2.0
3,3.0,2.0,0.0,-0.0,0.0,6.0,2.0,2.0,0.0,2.0,0.0
4,4.0,4.0,3.0,2.0,2.0,4.0,2.0,2.5,0.0,-0.0,0.0
5,5.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
6,6.0,3.0,3.0,3.0,5.5,7.0,4.0,4.0,0.0,1.0,0.0


## Myopic Market Clearing

In [11]:
# Run the market clearing
include.(("fct_split.jl","fct_mc_all.jl","fct_data_all.jl"))
S_end = zeros(maximum(G_df.MC)) # Set the penalty to zero (no penalty)
E_end = [false, false] # No final level
df_MCS = split(∆t, E, E_0, η_C, η_D, P_C, P_D, G_df, L_df, E_end, S_end)

SW_tot: 32.0
surplus_gen_tot: [5.0]
surplus_gen_sum_tot: 5.0
surplus_load_tot: [27.0]
surplus_stg_tot: 0.0


Row,Time,Market_Price,C1,p1,P1,U1,d1,D1,p_C,p_D,e
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,2.0,2.0,2.0,2.0,2.5,5.0,1.0,1.0,1.0,0.0,2.0
3,3.0,2.0,0.0,-0.0,0.0,6.0,2.0,2.0,0.0,2.0,0.0
4,4.0,4.0,3.0,2.0,2.0,4.0,2.0,2.5,0.0,-0.0,0.0
5,5.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
6,6.0,3.0,3.0,3.0,5.5,7.0,4.0,4.0,0.0,1.0,0.0


## Market Clearing with Virtual Linking Bids

In [12]:
# Set the level for the storage system at the end of each market interval:
E_end = [0.5, 2.5]
# Set a discount between 0 and 1 for the value of the stored energy (0: no discount)
discount = 0.0
;

In [13]:
# Initial storage values
E_init_inter = [0.0] # Level of storage initialized to 0 (storage empty)
S_init = [0.0] # Value of stored energy initialized to 0 (storage empty)

include.(("fct_vlb.jl","fct_mc_vlb.jl","fct_stor_val_update.jl","fct_data_all.jl"))
df_VLB = VLB_no_bin(∆t, E, η_C, η_D, P_C, P_D, G_df, L_df, E_end, E_init_inter, S_init, discount)

S: [2.0]
E_init: [0.5]
S: [2.0, 3.0]
E_init: [0.5, 2.0]
SW_tot: 25.5
surplus_gen_tot: [3.0]
surplus_gen_sum_tot: 3.0
surplus_load_tot: [21.5]
surplus_stg_tot: 1.0

In the inter-storage at the end of the test period: 
S: [2.0, 3.0]
E_init: [0.5, 2.0]


Row,Time,Market_Price,C1,p1,P1,U1,d1,D1,p_C_intra,p_D_intra,e_intra,p_D_inter,e_inter
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.0,0.0
2,2.0,2.0,2.0,2.5,2.5,5.0,1.0,1.0,1.5,0.0,2.5,0.0,-0.0
3,3.0,6.0,0.0,-0.0,0.0,6.0,2.0,2.0,0.0,2.0,0.5,-0.0,0.0
4,4.0,3.0,3.0,2.0,2.0,4.0,2.5,2.5,0.0,0.5,-0.5,0.0,0.5
5,5.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0,0.5
6,6.0,3.0,3.0,5.5,5.5,7.0,4.0,4.0,1.5,0.0,2.0,0.0,0.5
